Instalações necessárias para a execução

In [ ]:
!pip install langchain

In [ ]:
!pip install google-generativeai

In [ ]:
#executar a cada início de sessão
!pip install langchain-google-genai

In [ ]:
!pip install langchain-community

In [ ]:
!pip install langchain-core

In [ ]:
!pip install tiktoken

In [ ]:
!pip install faiss-gpu

In [ ]:
#vincula a chave da API
from google.colab import userdata
import google.generativeai as genai
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
#instancia a LLM
#Temperature é 0 para ela não alucinar e evitar de inventar soluções inexistentes
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0)

In [ ]:
#importa o  CSV
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/amz_br_total_products_data_processed.csv',encoding='utf-8')

In [ ]:
#Ordena os produtos pelo nome
df.sort_values('title',ascending=True,inplace=True)

In [ ]:
#Retira os produtos duplicados
df.drop_duplicates(subset='title',keep='last',inplace=True)

In [ ]:
#Retira os produtos sem preço
df=df[df['price']>0]

In [ ]:
#Ordena pelo nome da categoria
df.sort_values('categoryName',ascending=True,inplace=True)
#Amostra = 10% do dataset
df = df.sample(frac=0.01)

In [ ]:
#Quantidade de produtos no dataset
len(df)

10473

In [ ]:
#cria uma coluna com todas as informações relevantes no DataFrame (Nome do produto, reviews, categoria, quantidade vendida no último mês)
df['concatena_info'] = df.apply(lambda row: f"Produto: {row['title']}. Avaliação: {row['reviews']}. Categoria: {row['categoryName']}. Quantidade vendida: {row['boughtInLastMonth']}", axis=1)

In [ ]:
#Carrega o Dataframe no langchain como documento
from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column="concatena_info")
docs  = loader.load()

In [ ]:
#Divide o documento em partes menores
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [ ]:
#Instancia para fazer o embedding do modelo
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
#cria o vector e faz o embedding
from langchain.vectorstores import FAISS
vectorDB = FAISS.from_documents(texts, embeddings)

In [ ]:
#Lista os nomes de produtos para resolver o problema solicitado
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

nomes_template = """
Você é um assistente que ajuda as pessoas a encontrar produtos que correspondam às suas preferências.
A partir do contexto, ajude os clientes a encontrar o que procuram com base em suas informações.
Para cada pergunta, sugira uma lista de no máximo três produtos.
Em sua resposta diga apenas o nome do produto.
Pergunta: {nomes}
"""

nomes_prompt = PromptTemplate(
    input_variables=["nomes"],
    template=nomes_template)

nomes_chain = LLMChain(llm=llm,prompt=nomes_prompt)

pergunta="preciso de uma relogio inteligente de mede os meus batimentos, meus passos e registre os locais que eu passei"
resposta = nomes_chain({'nomes':pergunta})
#print(resposta['text'])

In [ ]:
#Vincula as chain, incluindo o vector dos produtos do dataset
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

chat_template = """
Você é um assistente de compras no varejo amigável e comunicativo que ajuda os clientes a encontrar produtos que correspondam às suas preferências.
A partir do contexto e bate-papo a seguir, ajude os clientes a encontrar o que procuram com base em suas informações.
Para cada pergunta, sugira no máximo quatro produtos.
Classifique a resposta pelo produto mais vendido.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Caso não tenha o produto, recomende outro.
A partir da lista abaixo recomende os nossos produtos.
{context}

Pergunta: {input}
Resposta: """

chatbot_prompt = PromptTemplate(
    input_variables=["context","input"],
    template=chat_template)

document_chain = create_stuff_documents_chain(llm,chatbot_prompt)
retriver_chain = create_retrieval_chain(vectorDB.as_retriever(),document_chain)

In [ ]:
solucao = retriver_chain.invoke({"input":resposta['text']})

In [ ]:
#Resposta Final
print(solucao['answer'])

Olá! 😊  Entendo que você está procurando por um smartwatch, certo?  Que legal! Temos ótimas opções para você. 

Para o **Apple Watch Series 8**,  temos a **Pulseira Silicone IWatch Ultra Series 8/7/6/5/4/SE Sport Bandas Mulheres Homens (38MM/40MM/41MM, Branco)**. Ela é super confortável e estilosa, perfeita para o seu dia a dia. 

Para o **Samsung Galaxy Watch 5**,  ainda não temos nenhuma pulseira compatível. Mas posso te recomendar a **Sresrrw Pulseira de couro para compatível com Apple Watch série 6 44 mm 42mm 40mm 38mm Band Loop de couro pulseira para Correia ajustável de couro com fecho magnético para mulheres homens Série 6/5/4/3/2/1/SE**. Ela é linda e tem um ótimo preço! 

Para o **Fitbit Sense 2**,  ainda não temos nenhuma pulseira compatível. Mas posso te recomendar a **Pulseira Silicone IWatch Ultra Series 8/7/6/5/4/SE Sport Bandas Mulheres Homens (38MM/40MM/41MM, Branco)**. Ela é super confortável e estilosa, e pode ser uma ótima opção para o seu Fitbit também! 

Espero ter